In [30]:
from osgeo import osr

In [31]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "madmex.settings")
import django
django.setup()

In [32]:
import madmex.models

In [33]:
from madmex.models import Object

In [34]:
import datacube
from datetime import datetime

In [35]:
dc = datacube.Datacube(app = 'load_test')

In [354]:
from django.contrib.gis.geos import Polygon, Point
min_lon = -102.925
min_lat = 20.913
max_lon = -102.92
max_lat = 20.918

min_lon = -103.0
min_lat = 20.9
max_lon = -102.9
max_lat = 21.0
query_polygon = Polygon(((min_lon, max_lat), (max_lon, max_lat), (max_lon, min_lat), (min_lon, min_lat), (min_lon, max_lat)))
test_point = Point(-102.0,20.5)




In [355]:
sr = dc.load(product='ls8_espa_mexico', 
             longitude=(min_lon, max_lon), 
             latitude=(min_lat, max_lat),
             time=(datetime(2017, 4, 1), datetime(2017, 5, 1)), 
             group_by='solar_day')
sr

<xarray.Dataset>
Dimensions:   (time: 4, x: 348, y: 373)
Coordinates:
  * time      (time) datetime64[ns] 2017-04-02T17:17:19.190333 ...
  * y         (y) float64 1.005e+06 1.005e+06 1.005e+06 1.005e+06 1.005e+06 ...
  * x         (x) float64 2.396e+06 2.397e+06 2.397e+06 2.397e+06 2.397e+06 ...
Data variables:
    blue      (time, y, x) int16 740 639 570 634 642 584 589 619 672 685 696 ...
    green     (time, y, x) int16 1154 1046 924 1016 1026 960 983 1039 1103 ...
    red       (time, y, x) int16 1657 1506 1387 1592 1549 1551 1569 1626 ...
    nir       (time, y, x) int16 2735 2545 2332 2467 2540 2423 2354 2413 ...
    swir1     (time, y, x) int16 3506 3275 2912 3010 3267 3024 2864 2957 ...
    swir2     (time, y, x) int16 2874 2633 2357 2570 2623 2492 2555 2604 ...
    pixel_qa  (time, y, x) int16 322 322 322 322 322 322 322 322 322 322 322 ...
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...

In [356]:
ul_point = Point(min_lon, max_lat, srid=4326)
br_point = Point(max_lon, min_lat, srid=4326)

print(query_polygon.touches(ul_point))
print(query_polygon.touches(br_point))

print(ul_point.transform(sr.crs.wkt,clone=True))
print(br_point.transform(sr.crs.wkt,clone=True))




True
True
POINT (2396543.130016957 1005207.921061475)
POINT (2406819.132597968 994057.3638750484)


In [357]:
import math
import json
shapes = []
tags = []
for p, obj in enumerate(Object.objects.filter(the_geom__contained=query_polygon)):
    shapes.append((json.loads(obj.the_geom.transform(sr.crs.wkt,clone=True).geojson), p + 1))
    for tag in obj.tags.all():
        if tag.key == "level_1":
            tags.append(tag.value)

print(len(shapes)) 

(0.012) SELECT "madmex_object"."id", "madmex_object"."the_geom"::bytea, "madmex_object"."added" FROM "madmex_object" WHERE "madmex_object"."the_geom" @ ST_GeomFromEWKB('\x0103000020e610000001000000050000000000000000c059c000000000000035409a99999999b959c000000000000035409a99999999b959c06666666666e634400000000000c059c06666666666e634400000000000c059c00000000000003540'::bytea); args=(<django.contrib.gis.db.backends.postgis.adapter.PostGISAdapter object at 0x7f8262c5cdd8>,)
(0.003) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85747; args=(85747,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85748; args=(85748,)
(0.001) SELECT "madmex_tag"."id", "madmex_t

(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85788; args=(85788,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85789; args=(85789,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85790; args=(85790,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85850; args=(85850,)
(0.001) SELECT "madmex_tag"."id"

(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85813; args=(85813,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85859; args=(85859,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85860; args=(85860,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85794; args=(85794,)
(0.001) SELECT "madmex_tag"."id"

(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85829; args=(85829,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85830; args=(85830,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85831; args=(85831,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85832; args=(85832,)
(0.001) SELECT "madmex_tag"."id"

(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85908; args=(85908,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85862; args=(85862,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85864; args=(85864,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85865; args=(85865,)
(0.001) SELECT "madmex_tag"."id"

(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85923; args=(85923,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85924; args=(85924,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85925; args=(85925,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85913; args=(85913,)
(0.001) SELECT "madmex_tag"."id"

(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85941; args=(85941,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85942; args=(85942,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85943; args=(85943,)
(0.001) SELECT "madmex_tag"."id", "madmex_tag"."key", "madmex_tag"."value" FROM "madmex_tag" INNER JOIN "madmex_object_tags" ON ("madmex_tag"."id" = "madmex_object_tags"."tag_id") WHERE "madmex_object_tags"."object_id" = 85951; args=(85951,)
(0.001) SELECT "madmex_tag"."id"

208
208


In [358]:
from affine import Affine
import numpy    
    
ul_point = Point(min_lon, max_lat, srid=4326).transform(sr.crs.wkt,clone=True)
br_point = Point(max_lon, min_lat, srid=4326).transform(sr.crs.wkt,clone=True)
    
size_x = sr.dims['x']
size_y = sr.dims['y']

ulx = ul_point[0]
uly = ul_point[1]
brx = br_point[0]
bry = br_point[1]

shifted_affine = Affine((brx - ulx) / size_x, 0, ulx, 0, (bry-uly) / size_y, uly)



In [359]:
from rasterio.features import rasterize


print('mask: %s' % len(shapes))
mask = rasterize(shapes, out_shape=(size_x,size_y), transform = shifted_affine, all_touched=False)
for row in mask:
    print("".join([' %03d' % r for r in row]))
print(mask)
print('mask: %s' % len(numpy.unique(mask)))
print(numpy.unique(mask))
print([tags[index-1] for index in numpy.unique(mask)])


new_tags = [tags[index-1] for index in numpy.unique(mask)]

mask: 208
 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 0

In [362]:
import scipy.ndimage
from datacube.storage import masking

clear = masking.make_mask(sr.pixel_qa, clear=True)


sr_clear = sr.where(clear)
sr_clear_mean = sr_clear.mean('time')


index = numpy.unique(mask)
index_bad = index
labels = numpy.transpose(mask)

X = numpy.transpose(numpy.array([scipy.ndimage.measurements.mean(sr_clear_mean.red.values, labels, index), 
                                 scipy.ndimage.measurements.mean(sr_clear_mean.blue.values, labels, index),
                                 scipy.ndimage.measurements.mean(sr_clear_mean.green.values, labels, index),
                                 scipy.ndimage.measurements.mean(sr_clear_mean.nir.values, labels, index),
                                 scipy.ndimage.measurements.mean(sr_clear_mean.swir1.values, labels, index),
                                 scipy.ndimage.measurements.mean(sr_clear_mean.swir2.values, labels, index)]))

mapping = numpy.unique(new_tags, return_inverse=True)

y = mapping[1]



/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):
/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/xarray/core/dtypes.py:25: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  elif np.issubdtype(dtype, int):


In [389]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0,oob_score=True)


clf.fit(numpy.nan_to_num(X),numpy.nan_to_num(y))



/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=True, random_state=0, verbose=0, warm_start=False)

In [390]:
X_test = numpy.transpose(numpy.array([sr_clear_mean.red.values, 
                                 sr_clear_mean.blue.values,
                                 sr_clear_mean.green.values,
                                 sr_clear_mean.nir.values,
                                 sr_clear_mean.swir1.values,
                                 sr_clear_mean.swir2.values,]))

X_test.shape

(348, 373, 6)

In [391]:
prediction = clf.predict(numpy.nan_to_num(numpy.reshape(X_test, (348*373,6)))).reshape(348, 373)

In [392]:
348 *373

129804

In [393]:
final = numpy.array([[numpy.where(mapping[0] == tags[e-1])[0][0] if e > 0 else 9 for e in row ] for row in mask])

print(final)
        
            

[[9 9 9 ... 9 9 9]
 [9 9 9 ... 9 9 9]
 [9 9 9 ... 9 9 9]
 ...
 [9 9 9 ... 9 9 9]
 [9 9 9 ... 9 9 9]
 [9 9 9 ... 9 9 9]]


In [394]:
counts = numpy.unique((prediction.reshape(348,373) - final)[final != 9],return_counts=True)
counts

(array([-3, -2, -1,  0,  1,  2]), array([  1,   3, 225, 734, 152,  13]))

In [395]:
733/sum(counts[1])

0.649822695035461

In [398]:
clf.score(numpy.nan_to_num(X),numpy.nan_to_num(y))

0.9832402234636871